# Applying Analysis Classes

An *Analysis* class can be applied to a dataset in a flexible manner, such as how and where the data is stored, which derivatives are required, and the computing environment in which to generate the derivatives. This is all specified by the arguments used to when "instantiating" Analysis object from the given Analysis class. The clear separation between design (classes) and application (objects) makes analyses implemented in Arcana highly portable between computing environments/research centres.

## Inspecting the Analysis Class

We will start by importing a predefined Analysis class `example.analysis.BasicBrainAnalysis`, which performs the same analysis as the workflow in the [Workflows Notebook](basic_workflow.ipynb). We print the "menu", the list of inputs, derivatives and parameters objects of this class can receive/derive, using the `static_menu` class method.

In [ ]:
from example.analysis import BasicBrainAnalysis
print(BasicBrainAnalysis.static_menu())

To see the "full" menu, which includes all the intermediate derivatives that can be produced by the analysis, pass the 'full' flag to `static_menu`

In [ ]:
print(BasicBrainAnalysis.static_menu(full=True))

## Defining the Dataset to Analyse

Arcana implicitly handles a lot of the menial tasks involved with data input/outputs such as file format conversions and inserting/retrieving data from a repository service (e.g. XNAT). To specify where your data is you need to create a Dataset object.

### Datasets in Directories on Local System

The simplest form of dataset object is just a directory on (or mounted on) your local file system. The structure of this directory depends on its "depth", i.e. whether it has multiple subjects and visits in it or not.

#### Depth: 0

Typically, just used for prototyping, but you can define a dataset for a single subject by just storing all the data within a single directory.

In [ ]:
%%bash
# Create a dataset for a single session in a flat directory. We will copy data from the BIDS formatted ds000114
SAMPLE_DSET=output/sample-datasets/depth0
mkdir -p $SAMPLE_DSET
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/
tree $SAMPLE_DSET

In [ ]:
from arcana import Dataset
dset0 = Dataset('output/sample-datasets/depth0')
print(dset0)

Notice the `depth` of this dataset is `0`. This means that there aren't any sub-directories for separate subjects or visits in it. However, all datasets in Arcana have an implicit depth of 2 (although future versions may relax this restriction) so we can see that the single "session" (a single visit of a subject) is assigned default subject and visit IDs of 'SUBJECT' and 'VISIT' respectively.

In [ ]:
print('subjects:', list(dset0.subject_ids))
print('visits:', list(dset0.visit_ids))

#### Depth: 1

For a multi-subject dataset we can add sub-directories for each subject

In [ ]:
%%bash
# Create a dataset for a multiple subjects in separate sub-directories by copying data from the BIDS formatted ds000114
SAMPLE_DSET=output/sample-datasets/depth1
mkdir -p $SAMPLE_DSET/sub1 $SAMPLE_DSET/sub2  $SAMPLE_DSET/sub3
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1
find data/ds000114/sub-02/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2
find data/ds000114/sub-03/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3
tree $SAMPLE_DSET

In [ ]:
dset1 = Dataset('output/sample-datasets/depth1', depth=1)
print(dset1)
print('subjects:', list(dset1.subject_ids))
print('visits:', list(dset1.visit_ids))

**Note** that we need to explicitly provide the depth of `1` otherwise Arcana will interpret our 'sub1', 'sub2' and 'sub3' as filesets.

#### Depth: 2

For a dataset with multiple visits per subject we use a `depth == 2`

In [ ]:
%%bash
# Create a dataset for a multiple subjects in separate sub-directories by copying data from the BIDS formatted ds000114
SAMPLE_DSET=output/sample-datasets/depth2
mkdir -p $SAMPLE_DSET/sub1/test $SAMPLE_DSET/sub1/retest $SAMPLE_DSET/sub2/test $SAMPLE_DSET/sub2/test \
         $SAMPLE_DSET/sub2/retest $SAMPLE_DSET/sub3/test $SAMPLE_DSET/sub3/retest
find data/ds000114/sub-01/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1/test
find data/ds000114/sub-02/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2/test
find data/ds000114/sub-03/ses-test -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3/test
find data/ds000114/sub-01/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub1/retest
find data/ds000114/sub-02/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub2/retest
find data/ds000114/sub-03/ses-retest -name '*.nii.gz' | xargs -I% cp -f % $SAMPLE_DSET/sub3/retest
tree $SAMPLE_DSET

In [ ]:
dset2 = Dataset('output/sample-datasets/depth2', depth=2)
print(dset2)
print('subjects:', list(dset2.subject_ids))
print('visits:', list(dset2.visit_ids))

### Filtering Subjects and Visits

Just say the `retest` session of `Subject 3` was corrupted or was missing a scan we require for our analysis, we can exclude it from the analysis by either dropping `Subject 3` or `retest` from the dataset by filtering the IDs with the `subject_ids` and/or `visit_ids` parameters.

In [ ]:
dset2_filter_subs = Dataset('output/sample-datasets/depth2', depth=2, subject_ids=['sub1', 'sub2'])
print(dset2_filter_subs)
print('subjects:', list(dset2_filter_subs.subject_ids))
print('visits:', list(dset2_filter_subs.visit_ids))

To filter the visits used in the analysis

In [ ]:
dset2_filter_vis = Dataset('output/sample-datasets/depth2', depth=2, visit_ids=['test'])
print(dset2_filter_vis)
print('subjects:', list(dset2_filter_vis.subject_ids))
print('visits:', list(dset2_filter_vis.visit_ids))

Or to filter both

In [ ]:
dset2_filter_both = Dataset('output/sample-datasets/depth2', depth=2, subject_ids=['sub1', 'sub2'], visit_ids=['test'])
print(dset2_filter_both)
print('subjects:', list(dset2_filter_both.subject_ids))
print('visits:', list(dset2_filter_both.visit_ids))

### Datasets on XNAT

In addition to data stored on your local file system, Arcana can transparently handle all interactions (i.e. downloading/uploading) with datasets stored in XNAT repositories.

To test this we will use a public project set up on Monash's public XNAT instance

In [ ]:
import os.path as op
from arcana import XnatRepo
xnat_repo = XnatRepo(server='https://xnat.monash.edu', cache_dir=op.expanduser('~/xnat-cache'))
print(xnat_repo)
xnat_dataset = xnat_repo.dataset('MISC0002')  # This is the ID of the project on MXNAT
print(xnat_dataset)
print('subjects:', list(xnat_dataset.subject_ids))
print('visits:', list(xnat_dataset.visit_ids))

**Note:** If you have a look at the 'MISC0002' project on https://xnat.monash.edu.au you will notice that subjects and sessions are labelled according to the conventions used at MBI, i.e. PROJECTID_SUBJECTID and PROJECTID_SUBJECTID_VISITID for subject and session IDs, respectively. This is a current limitation of Arcana although it should be relaxed in the next month or so.

### Notes on Other Repository/Dataset Types

At this stage XNAT is the only data repository platform supported by Arcana. However, care has been taken to modularise the code as much as possible so it should be fairly straightforward to implement support for other platforms (e.g. Loris, DaRIS, MyTaRDIS) as long as they have a REST API (or equivalent) that enables you to list, get and put data. See the base repository class `arcana.repository.base.Repository` for details on the six abstract methods that need to be overriden. 

Unimelb users (hello David:), there used to be a DaRIS module in early versions of Arcana, which could be ressurected without too much effort if you have a DaRIS instance to test against.

`Banana` also adds support for the [BIDS](https://bids.neuroimaging.io) format via the `BidsDataset`. The BidsDataset objects are able to parse the specific naming conventions and directory-tree structure that BIDS requires, and insert derivatives at in the `derivatives` directory.


### Components of a Dataset: Filesets and Fields

A `Dataset` object represents data in a hierarchical tree consisting of nodes for each session, subject, visit and the dataset as a whole. Each node contains sets of `Fileset` and `Field` objects. `Fileset` represents either a single file, a primary file plus headers and/or side-car files, or a directory with arbitrary contents. A `Field` is used to store boolean, integer, floating point or character strings, or small arrays thereof (e.g. echo times). Where fields are stored depends on the repository type, on the local FS they are stored in JSON files, and on XNAT they are stored in "custom variables" of the session.

Each data item (i.e. fileset or field), is assigned a "frequency", which refers to where it sits in the tree hierarchy. Valid frequencies are 'per_session' (i.e. every session contains a unique version of the item), 'per_visit' (there is one item per visit), 'per_subject' (there is one item per subject) and 'per_dataset'(there is only one in the whole dataset). Filesets are associated with a `FileFormat` object (e.g. `nifti_gz_format`, `dicom_format`), which determines how the data is treated (e.g. whether it is a directory or has associated header files).

## Configuring the Software Environment

A key feature of Nipype and Arcana is the ability to interface with any sort of external toolkit, whether it runs in Bash, Python, Matlab, etc... For the sake of reproducibility, it is important to detect and record exactly which version of these tools was used to run the analysis.

While it is often advisable to use the latest versions of such toolkits, in some circumstances you may need to use different versions of the same package (e.g. FSL, SPM) to run different sections of your workflow (a real headache). In order to manage the installation and use of different toolkits versions on the same system, high-performance computing clusters (such as MASSIVE/CVL) typically require ["Environment Modules"](http://modules.sourceforge.net) to be loaded before running a toolkit.

Arcana encapsulates the handling of such issues within `Environment` objects that are configured by the user at runtime. There are currently two available classes, `StaticEnv` and `ModulesEnv`.

### Static Environments

As the name suggests, `StaticEnv` does not attempt to configure the software environment on the system, and simply detects and records the version of the software used. Because of this inflexibility it is typically used in the prototyping phase but could still be useful when running on local workstations without environment modules installed.

Configuring a static environment is very simple as they only take two fairly self-explanatory parameters, `fail_on_missing` and `fail_on_undetectable` (both `True` by default). Therefore it is typically okay to just initialise the class without any parameters, i.e.


In [ ]:
from arcana import StaticEnv
static_env = StaticEnv()
print(static_env)

### Modules Environments

`ModulesEnv` loads and unloads environment modules on the system before and after each node of the workflow is run, respectively, based on software requirements specified in the Analysis class (see the `arcana.environment.requirement` and `banana.requirement` packages for examples).

In most cases, environment modules are named fairly sensibly and line up with the names of the built-in requirements of Arcana and Banana. But in the case of unconvential naming schemes, Arcana/Banana requirements can be mapped onto the names and versions of modules installed on your system with the `packages_map` and `versions_map` parameters.

For example, on the CVL there is a special version of Matlab 2017b which is interacts with the machine learning package "caffe" that has the version 'matlab/r2017b-caffe' is used in place of 2017b

In [ ]:
from arcana import ModulesEnv
from arcana.environment.requirement import matlab_req
modules_env = ModulesEnv(versions_map={matlab_req: {'2017b-caffe': '2017b'}})
print(modules_env)

**Note:** when an analysis workflow is run using `ModulesEnv` it will unload previously loaded versions of modules (such as those in the `neuro-workshop` module on the CVL).

### Notes on Other Environment Types

You can't talk about reproducibility in science without mentioning container technology these days (for good reason) so you may ask why Arcana/Banana doesn't use Docker or Singularity to manage software versions. The answer is just that I haven't found the time to implement this yet, but it is a very high priority for this summer :)

The plan is to create a new `Environment` modules for both [Docker](http://docker.com) and [Singularity](https://singularity.lbl.gov) that will manage the installation of appropriate images on the system as well as running each node of the workflow in a separate container. I expect that these modules will become the standard way to run Arcana (outside of prototyping) in the future.

The workaround at present is to use the `monashbi/banana` Docker container, which has all the relevant tools installed within a single container. However, you cannot use this on the CVL as Docker is not installed for security reasons so you are stuck with the `ModulesEnv` (for now).

## Workflow Execution with Processors

`Processor`s in Arcana are very much like the concept of execution plugins in Nipype (see the [Excecution Plugins Notebook](basic_plugin.ipynb)). In fact they are just very thin wrappers around execution plugins plus a couple of methods implemented in the base `Processor` class. As such, they control how the workflow graph is executed on the host system (i.e. single process, multi-process, via a job-scheduler). For the user the only important difference is they also specify the working directory and a few other execution parameters used when running the analysis. There are only three processors currently implemented in Arcana `SingleProc`, `MultiProc` and `SlurmProc`, but wrapping up other Nipype plugins is trivial if there is another you require.

`SingleProc` does what the name suggests, runs the workflow in a single process. It wraps Nipype's `LinearPlugin` plugin, and any unused keyword arguments will be passed to `LinearPlugin`. To instantiate it you only need to provide the working directory:

In [ ]:
from arcana import SingleProc
single_proc = SingleProc('work/arcana-example')
print(single_proc)

Likewise, `MultiProc` wraps up the `MultiProcPlugin`. It only has one additional parameter `num_processes`, which you can omit to use all available processes on your system

In [ ]:
from arcana import MultiProc
multi_proc = MultiProc('work/arcana-example', num_processes=4)
print(multi_proc)

`SlurmProc` wraps the `SlurmGraphPlugin`, and is what you want to use when running jobs on MASSIVE or other high-performance computing clusters with SLURM schedulers. To run on MASSIVE you will need to provide values for the `partition`, `account` and `email`, e.g.

In [ ]:
from arcana import SlurmProc
slurm_proc = SlurmProc('/scratch/fs97/your-analysis', account='your-account', email='your.email@monash.edu',
                       partition='m3a')

### Reprocessing

With Arcana you can run analyses on large datasets (or small if you like) in an iterative manner as data is acquired. Arcana saves all intermediate derivatives back into the repository for reuse along with detailed provenance, such as parameters supplied to each node and the versions of external toolkits used.

At the start of each run the provenance is checked to see if the workflows that will be run on the software match those that were run previously and will raise a `ArcanaReprocessException` if there any differences. In this case you have two options, reprocess the nodes that differ from the workflow you are trying to run now (recommended), or if the differences are spurious and you are sure that they won't make a difference you can ignore them. To reprocess the data, simply provide `reprocess` flag to the Processor initialisation, i.e.


In [ ]:
single_proc_reprocess = SingleProc('work/arcana-example', reprocess=True)


Ignoring elements of the provenance is a little trickier and requires paths to be provided to the `prov_check` and/or `prov_ignore` parameters. `prov_check` and `prov_ignore` represent regular expressions of paths to elements provenance JSON file. Paths in `prov_check`, including all sub-paths, will be included unless they are also in `prov_ignore`.

By default the paths for these parameters are

```python
DEFAULT_PROV_CHECK = ['workflow', 'inputs', 'outputs', 'joined_ids']
DEFAULT_PROV_IGNORE = ['.*/pkg_version',
                       'workflow/nodes/.*/requirements/.*/local_version',
                       'workflow/nodes/.*/requirements/.*/local_name']
```

which aren't recommeded to be altered with much. So the best bet is to append the path you want to ignore, which should be returned in the reprocess exception text to `DEFAULT_PROV_IGNORE`, e.g.


In [ ]:
single_proc_ignore = SingleProc('work/arcana-example',
                                prov_ignore=SingleProc.DEFAULT_PROV_IGNORE
                                + ['workflow/nodes/.*/requirements/fsl/version'])

## Creating the Analysis Object

Once you have your `Dataset`, `Environment` and `Processor` objects configured you are ready to instantiate the `Analysis` class. All that is left is to provide the parameters you want to use for the analysis and map the data in your dataset to the inputs of the Analysis class.

### Input Filters

The first step is map the inputs of the Analysis class onto your data. Each input should map onto a "slice" of equivalent data items across dataset (i.e. for every session for 'per_session' items, for every subject for 'per_subject' items, etc...). Depending on how your dataset was acquired the names used for equivalent scans between subjects and sessions should be similar (hopefully the same). However, for some projects (e.g. collected by different radiographers at different sites for example) you might not be so lucky and you might need to use several criteria to select the right scans for each session.

Such criteria are specified in `FilesetFilter` and `FieldFilter` objects. The filter must select exactly one data item (fileset or field) for each node in the slice. If there is a node without an equivalent item it should be excluded from the dataset (see the "Filtering Subjects and Visits" section).

In the easiest case where you have consistent naming conventions across your dataset the "filters" can be specified in a simple str->str dictionary, e.g.

In [ ]:
inputs = {
    'magnitude': 'R-L ep2d_diff_mrtrix_33_dir_3_inter_b0_p2_RL',
    'reverse_phase': 'PRE DWI L-R distortion correction 36 DIR MRTrix'}

**Note:** File extensions are stripped from the names files when they are read into a `Dataset` object, so if you add file extensions to the end of your name pattern it won't work!

But just say that the names of the reverse phase images 'distortion' within their names but otherwise vary quite a bit we provide a list of `FilesetFilter` objects and use regular expressions to match all scans with 'distortion' in their names.

In [ ]:
from arcana import FilesetFilter
inputs = [
    FilesetFilter('magnitude', 'R-L ep2d_diff_mrtrix_33_dir_3_inter_b0_p2_RL'),
    FilesetFilter('reverse_phase', '.*distortion.*', is_regex=True)]

There are a number of other criteria that can be used to differentiate data within the datset, which you can see with

In [ ]:
help(FilesetFilter)

#### Some Useful Things to Note About Inputs

* You can provide "inputs" to any data item (i.e. not parameter) in the Analysis class' menu, not just the ones marked as Inputs.

* Input filesets don't need to be in the same format as the one expected by the Analysis class as long as a converter exists between them. Arcana will automatically insert a converter node under the hood and handle this all for you!

### Parameterising

Parameters are defined analysis-wide, i.e. not for each workflow, and provided to the Analysis object with it is initialised. Each parameter is created with a recommended default value so in many cases you won't need to provide any explicit parameters. The parameters you need to change can be provided in a simple dictionary

In [ ]:
parameters = {
    'response_algorithm': 'dhollander',
    'global_tracks_cutoff': 0.05}

### Putting it All Together

Now we have all the pieces we require and we just need to give the Analysis object a name. This name is used to differentiate different analyses performed on the same dataset (by saving them in different directories/sessions). This allows you to do multiple analyses side by side with different parameterisations (although be sure to correct your statistics for multiple comparisons!).

In [ ]:
my_analysis = BasicBrainAnalysis(
    'my_analysis',
    dataset=dset1,
    processor=single_proc,
    environment=static_env,
    inputs=[
        FilesetFilter('magnitude', '.*T1w$', is_regex=True)],
    parameters={
        'smoothing_fwhm': 5.0})

### Deriving Derivatives

Now that we have our Analysis object to play with we can start to generate derivatives by providing the names of the derivatives we want to the `derive` method

In [ ]:
my_analysis.derive('brain')

We can access the fileset-slice of smoothed images (i.e. for each subject) using the `data` method

In [ ]:
brain_slice = my_analysis.data('brain')
for fileset in brain_slice:
    print(fileset.path)

In [ ]:
! tree -D output/sample-datasets/depth1

So you can see that there is a `smooth.nii.gz` file in a sub-directory called 'my_analysis' in each subject. If we now want to generate a 'smooth_masked' image, it will be reused as long as the parameterisation isn't changed

In [ ]:
smooth_masked_slice = my_analysis.data('smooth_masked', derive=True)

Examining the contents of the dataset directory we can see that within the 'my_analysis' sub-directories are derivatives for brain, brain_mask, smooth and smooth masked, but that the time-stamps for the brain and brain_mask files are from the previous call to `derive`.

In [ ]:
! tree -D output/sample-datasets/depth1

Notice also the provenance files stored for each pipeline in the '__prov__' directory. If we take a look at the contents we can see a list of all the parameters and requirements used in the analysis, as well as checksums for all inputs and outputs of the pipeline.

In [ ]:
! cat output/sample-datasets/depth1/sub1/my_analysis/__prov__/smooth_mask.json

If we were to rerun the analysis (perhaps on a different day) with reinstantiated object with different parameters we will get a `ArcanaReprocessException`

In [ ]:
my_analysis = BasicBrainAnalysis(
    'my_analysis',  # The name needs to be the same as the previous version
    dataset=dset1,
    processor=single_proc,
    environment=static_env,
    inputs=[
        FilesetFilter('magnitude', '.*T1w$', is_regex=True)],
    parameters={
        'smoothing_fwhm': 6.0})  # Note the large smoothing kernel

In [ ]:
try:
    my_analysis.derive('smooth_masked')
except Exception as e:
    print(e)

But if we supply the `reprocess` flag it will overwrite only the derivatives that need to be reprocessed

In [ ]:
my_analysis.processor.reprocess = True
my_analysis.derive('smooth_masked')

In [ ]:
! tree -D output/sample-datasets/depth1

### Writing Methods to Produce Research Outputs

While you could just leave it at using Arcana to derive the derivatives you need for your analysis and go and produce your plots using Matlab like you usually do. If you want to go the extra mile, it would be cool to add methods to your to actually produce the results that you put in your paper/poster. As this should make it possible to share your complete analysis with another researcher in a way that is easy for them replicate on their own data.

For example, the `BasicBrainAnalysis` has a method called `plot_comparision` which uses the `data` method internally to access the derivatives for each subject and produce the plot from the Workflows notebook.

In [ ]:
my_analysis.plot_comparision()

## Exercise 1

In the `example.analysis` module there is another Analysis class called `ToyAnalysis` that uses standard unix tools 'awk', 'grep' to extract a simple metrics from a file. Given the dataset created in `output/sample-datasets/toy-dataset` by the cell below, use `ToyAnalysis` to derive and print the average weight across all subjects and visits.

In [ ]:
import os.path
import shutil
import numpy

# Create a test project with dummy data
NUM_SUBJECTS = 3
NUM_VISITS = 2

HEIGHT_MEAN = 1700
HEIGHT_STD = 150
WEIGHT_MEAN = 70
WEIGHT_STD = 25
HEAD_CIRC_MEAN = 570
HEAD_CIRC_STD = 30

subjects = ['subject{}'.format(i) for i in range(NUM_SUBJECTS)]
visits = ['visit{}'.format(i) for i in range(NUM_VISITS)]

project_dir = os.path.abspath('output/sample-datasets/toy-datasets')
# Clean old directory
shutil.rmtree(project_dir, ignore_errors=True)
os.mkdir(project_dir)
for subj in subjects:
    for visit in visits:
        session_dir = os.path.join(project_dir, subj, visit)
        try:
            os.makedirs(session_dir)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        with open(os.path.join(session_dir, 'metrics.txt'), 'w') as f:
            f.write('height {}\n'.format(numpy.random.randn() * HEIGHT_STD + HEIGHT_MEAN))
            f.write('weight {}\n'.format(numpy.random.randn() * WEIGHT_STD + WEIGHT_MEAN))
            f.write('head_circ {}\n'.format(numpy.random.randn() * HEAD_CIRC_STD + HEAD_CIRC_MEAN))
print("Created project in {} directory".format(project_dir))

In [ ]:
! tree output/sample-datasets/toy-datasets

In [ ]:
! cat output/sample-datasets/toy-datasets/subject0/visit0/metrics.txt

In [ ]:
## Write your solution here

In [ ]:
from example.analysis import ToyAnalysis
print(ToyAnalysis.static_menu())

In [ ]:
weight_analysis = ToyAnalysis(
    'weight_analysis',  # The name needs to be the same as the previous version
    dataset=Dataset('output/sample-datasets/toy-datasets', depth=2),
    processor='work/toy-analysis',  # If you just provide a work dir then SingleProc is assumed
    inputs={'body_metrics': 'metrics'},
    parameters={'metric_of_interest': 'weight'})

In [ ]:
weight_analysis.derive('average')

In [ ]:
print(weight_analysis.data('average').value())

## Exercise 2

On the same dataset this time use `ToyAnalysis` to produce the standard deviation in head circumferences across the sessions

In [ ]:
## Write your solution here

In [ ]:
head_circ_analysis = ToyAnalysis(
    'head_circ_analysis',  # The name needs to be the same as the previous version
    dataset=Dataset('output/sample-datasets/toy-datasets', depth=2),
    processor='work/toy-analysis',  # If you just provide a work dir then SingleProc is assumed
    inputs={'body_metrics': 'metrics'},
    parameters={'metric_of_interest': 'head_circ'})

In [ ]:
head_circ_analysis.derive('std_dev')

In [ ]:
print(head_circ_analysis.data('std_dev').value())